# Stock Market Predictions - Time Series

In this project, we'll be working with data from the S&P500 Index. We'll be using historical time tagged data to make predictions about future prices. Data includes daily opening, highest of the day, lowest of the day and closeing indexes as well as volume of the days trade. Time frame is Jan 1, 1950 to Dec 7, 2015.

The columns of the dataset are:

- Date -- The date of the record.
- Open -- The opening price of the day (when trading starts).
- High -- The highest trade price during the day.
- Low -- The lowest trade price during the day.
- Close -- The closing price for the day (when trading is finished).
- Volume -- The number of shares traded.
- Adj Close -- The daily closing price, adjusted retroactively to include any corporate actions. Read more here.

Normal machine learning datasets are formed of independent incidents at each row. In this case, since the data is time-sequential, we should be extra careful not to inject future knowledge into past rows when doing the predictions. 

At time series problems, we should create indicators to make future predictions. When defining the values of this indicators, we should use the past data, but only the past not the data of the day since we cannot know any value of the day upfront when we predict future data.


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
df = pd.read_csv("sphist.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
Date         16590 non-null object
Open         16590 non-null float64
High         16590 non-null float64
Low          16590 non-null float64
Close        16590 non-null float64
Volume       16590 non-null float64
Adj Close    16590 non-null float64
dtypes: float64(6), object(1)
memory usage: 907.3+ KB


In [4]:
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117
3,2015-12-02,2101.709961,2104.270020,2077.110107,2079.510010,3.950640e+09,2079.510010
4,2015-12-01,2082.929932,2103.370117,2082.929932,2102.629883,3.712120e+09,2102.629883


It is nice to see that there is no null entries in the whole table.

In [5]:
#Convert the date column, which is string now, into datetime object
df.Date = pd.to_datetime(df.Date, format="%Y-%m-%d")

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16590 entries, 0 to 16589
Data columns (total 7 columns):
Date         16590 non-null datetime64[ns]
Open         16590 non-null float64
High         16590 non-null float64
Low          16590 non-null float64
Close        16590 non-null float64
Volume       16590 non-null float64
Adj Close    16590 non-null float64
dtypes: datetime64[ns](1), float64(6)
memory usage: 907.3 KB


Everything seem fine so far. We need to sort the ___Date___ in ascending order rather than descending.

In [7]:
df.sort_values("Date", inplace = True)

In [8]:
df.reset_index(inplace=True, drop=True)
df.head()

,Date,Open,High,Low,Close,Volume,Adj Close
0,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
1,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
2,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93
3,1950-01-06,16.98,16.98,16.98,16.98,2010000.0,16.98
4,1950-01-09,17.08,17.08,17.08,17.08,2520000.0,17.08


Many different type of indicators could be used. We prefered four of them:

- Average price over the past 5 days
- Average price over the past year
- Ratio of the two indicators up above
- Average volume over the past 5 days
- Standard deviation of the volume over the past 5 days

In order to measure the accuracy of the model, we prefered to use MSE.

In [9]:
from datetime import datetime


last_5 = []
last_365 = []
vol_last_5 = []
std_vol_last_5 = []

for index, row in df.iterrows():
    the_date = row.Date
    
    if index<5:
        last_5.append(0)
        vol_last_5.append(0)
        std_vol_last_5.append(0)
    else:
        last_5.append(df.Close[index-5:index].sum()/5)
        vol_last_5.append(df.Volume[index-5:index].sum()/5)
        std_vol_last_5.append(df.Volume[index-5:index].std())
        
    if row.Date<datetime(year=1951, month=1, day=3):
        last_365.append(0)
    elif (the_date.month == 2) & (the_date.day == 29):
        one_year_before = datetime(year=the_date.year - 1, month = the_date.month, day = the_date.day-1)
        l_365 = df[  (df["Date"]>one_year_before) &  (df["Date"]<the_date)  ]["Close"]
        last_365.append( l_365.sum() / len(l_365) )
        
    else:
        one_year_before = datetime(year=the_date.year - 1, month = the_date.month, day = the_date.day)
        l_365 = df[  (df["Date"]>one_year_before) &  (df["Date"]<the_date)  ]["Close"]
        last_365.append( l_365.sum() / len(l_365) )
   
    #print(the_date)
    
df["last_5"] = last_5
df["last_365"] = last_365
df["ratio"] = df["last_5"] / df["last_365"]
df["vol_last_5"] = vol_last_5
df["std_vol_last_5"] = std_vol_last_5
        
    

In [10]:
print(len(last_365))

16590


In [11]:
df.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,last_5,last_365,ratio,vol_last_5,std_vol_last_5
0,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,0.000,0.0,NaN,0.0,0.000000
1,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,0.000,0.0,NaN,0.0,0.000000
2,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,0.000,0.0,NaN,0.0,0.000000
3,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,0.000,0.0,NaN,0.0,0.000000
4,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,0.000,0.0,NaN,0.0,0.000000
5,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900,0.0,inf,2046000.0,529650.828377
6,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974,0.0,inf,2226000.0,298043.620968
7,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022,0.0,inf,2374000.0,272084.545684
8,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.988,0.0,inf,2458000.0,382452.611444
9,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.926,0.0,inf,2722000.0,446172.612337


In [12]:
df.ratio.value_counts().iloc[:5]

inf         245
1.143740      1
1.066487      1
1.164372      1
0.969572      1
Name: ratio, dtype: int64

In [13]:
def replace_inf (num):
    if num == np.inf:
        return np.nan
    else:
        return num

df["ratio"] = df["ratio"].apply(replace_inf)
df.head(10)

,Date,Open,High,Low,Close,Volume,Adj Close,last_5,last_365,ratio,vol_last_5,std_vol_last_5
0,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,0.000,0.0,NaN,0.0,0.000000
1,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,0.000,0.0,NaN,0.0,0.000000
2,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,0.000,0.0,NaN,0.0,0.000000
3,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,0.000,0.0,NaN,0.0,0.000000
4,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,0.000,0.0,NaN,0.0,0.000000
5,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900,0.0,NaN,2046000.0,529650.828377
6,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974,0.0,NaN,2226000.0,298043.620968
7,1950-01-12,16.760000,16.760000,16.760000,16.760000,2970000.0,16.760000,17.022,0.0,NaN,2374000.0,272084.545684
8,1950-01-13,16.670000,16.670000,16.670000,16.670000,3330000.0,16.670000,16.988,0.0,NaN,2458000.0,382452.611444
9,1950-01-16,16.719999,16.719999,16.719999,16.719999,1460000.0,16.719999,16.926,0.0,NaN,2722000.0,446172.612337


In [14]:
print (df.Date.min())
print (df.Date.max())

1950-01-03 00:00:00
2015-12-07 00:00:00


In [15]:
df = df[df["Date"] >= datetime(year=1951, month=1, day=3)]

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16340 entries, 250 to 16589
Data columns (total 12 columns):
Date              16340 non-null datetime64[ns]
Open              16340 non-null float64
High              16340 non-null float64
Low               16340 non-null float64
Close             16340 non-null float64
Volume            16340 non-null float64
Adj Close         16340 non-null float64
last_5            16340 non-null float64
last_365          16340 non-null float64
ratio             16340 non-null float64
vol_last_5        16340 non-null float64
std_vol_last_5    16340 non-null float64
dtypes: datetime64[ns](1), float64(11)
memory usage: 1.6 MB


We will use data up to 1st of Jan, 2013, to make predictions after this date.

In [17]:
train = df[df["Date"] < datetime(year=2013, month=1, day=1)]
test = df[df["Date"] >= datetime(year=2013, month=1, day=1)]

In [18]:
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression

In [19]:
lr = LinearRegression()
features = ["last_5", "last_365", "ratio", "vol_last_5", "std_vol_last_5"]
target="Close"

lr.fit(train[features], train[target])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [20]:
predictions = lr.predict(test[features])

In [21]:
mae = mean_absolute_error(test[target], predictions)

In [22]:
print(round(mae,2))

16.17


In [23]:
test.Close.mean()

1874.8903383897166

In [24]:
test.Close.std()

192.38905804320322

We predictied closing values with some error, which is quite good in comparison with the mean and the std of the test set target series.
### Project is over.